In [1]:
import os
from tqdm import tqdm
import argparse

import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("sample_train.ipynb"))))
from common.parser import yaml_parser
from common.recoder import save_checkpoint
from data.yolo_dataset import *
from model.MyYOLOv3 import YOLOv3Loss
from model.darknet2pytorch import DarknetParser

import torch
import torch.nn
from torch.utils.tensorboard import SummaryWriter



In [2]:

def train(
        model,
        train_loader,
        loss_func,
        optimizer,
        dataset_option,
        model_option,
        device,
        epoch,
        logger
        ):
    model.train()

    scales = torch.tensor(model_option["YOLOv3"]["SCALES"]).to(device)       ## [13, 26, 52]
    anchors = torch.tensor(model_option["YOLOv3"]["ANCHORS"]).to(device)

    for i, (batch_img, batch_label, batch_img_path) in enumerate(train_loader, 0):
        n_iteration = (optimizer_option["OPTIMIZER"]["ITERS_PER_EPOCH"] * epoch) + i

        batch_img = batch_img.to(device)
        batch_label = (batch_label[0].to(device), batch_label[1].to(device), batch_label[2].to(device))
        
        #################
        ##  FORWARDING ##
        #################
        pred = model(batch_img)                                                       ### batch_img: tensor(   N, 3, 608, 608) . . . . . . . . . . . N = batch_size
        pred = pred.data.cpu().numpy()
        loss = ( loss_func(pred[2], batch_label[0], scales[0], anchors=anchors[0])    ######## pred: tensor(3, N, 3, S, S, 1 + 4 + class_offset) . . S = scale_size
               + loss_func(pred[1], batch_label[1], scales[1], anchors=anchors[1])    # batch_label: tensor(3, N, 3, S, S, 1 + 4 + class_offset)
               + loss_func(pred[0], batch_label[2], scales[2], anchors=anchors[2]) )  ##### anchors: tensor(3,    3,       2) . . . is list of pairs(anch_w, anch_h)
        loss /= 3

        logger.add_scalar('train/loss', loss.item(), n_iteration)

        # print(f"loss: {loss}")

        #################
        ## BACKWARDING ##
        #################
        loss.backward()
        optimizer.step()

        torch.cuda.empty_cache()



In [3]:

def valid(
        model,
        valid_loader,
        model_option,
        device,
        epoch,
        logger
        ):
    model.eval()
    true_pred_num = 0
    gt_num = 0

    scales = torch.tensor(model_option["YOLOv3"]["SCALES"]).to(device)       ## [13, 26, 52]
    anchors = torch.tensor(model_option["YOLOv3"]["ANCHORS"]).to(device)

    for i, (batch_img, batch_label, batch_img_path) in enumerate(valid_loader, 0):
        batch_img = batch_img.to(device)
        batch_label = batch_label.to(device)
        
        pred = model(batch_img)

        ## Post-Processing?

        ## Get the number of both true predictions and ground truth


    ## Examine Accuracy
    acc = (true_pred_num / gt_num + 1e-16) * 100
    logger.add_scalar('test/acc', acc, epoch)

    return acc



In [4]:
# parser = argparse.ArgumentParser()

# parser.add_argument("--config", type=str, default="C:/Users/ryyoon/RY_GitHub/Lets_Pytorch/YOLO-v3/configs/darknet/yolov4.cfg")
# parser.add_argument("--weight", type=str, default="C:/Users/ryyoon/RY_GitHub/Lets_Pytorch/YOLO-v3/configs/darknet/yolov4.weights")

# parser.add_argument("--dataset", type=str, default="C:/Users/ryyoon/RY_GitHub/Lets_Pytorch/YOLO-v3/configs/dataset/yolo_dataset.yml")
# parser.add_argument("--model", type=str, default="C:/Users/ryyoon/RY_GitHub/Lets_Pytorch/YOLO-v3/configs/model/yolo_model.yml")
# parser.add_argument("--optimizer", type=str, default="C:/Users/ryyoon/RY_GitHub/Lets_Pytorch/YOLO-v3/configs/optimizer/optimizer.yml")

# parser.add_argument("--weight-save-dir", type=str, default="C:/Users/ryyoon/RY_GitHub/Lets_Pytorch/YOLO-v3/weights")

# args = parser.parse_args()

import easydict

args = easydict.EasyDict({
    "config": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov4.cfg",
    "weight": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov4.weights",
    "dataset": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/dataset/yolo_dataset.yml",
    "model": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/model/yolo_model.yml",
    "optimizer": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/optimizer/optimizer.yml",
    "weight_save_dir": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/weights"
})


dataset_option = yaml_parser(args.dataset)
model_option = yaml_parser(args.model)
optimizer_option = yaml_parser(args.optimizer)


In [5]:

######################
## BUILD DATALOADER ##
######################
# train_set_num, train_loader, _ = build_DataLoader(dataset_option, model_option, optimizer_option)

train_dataset = YoloDataset(dataset_option, model_option, split="valid")
train_loader = DataLoader(train_dataset, batch_size=optimizer_option["OPTIMIZER"]["BATCH_SIZE"], shuffle=True, collate_fn=collate_fn)
valid_dataset = YoloDataset(dataset_option, model_option, split="valid")
valid_loader = DataLoader(valid_dataset, batch_size=optimizer_option["OPTIMIZER"]["BATCH_SIZE"], shuffle=True, collate_fn=collate_fn)

In [6]:

device = torch.device('cpu')
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###########################
## BUILD MODEL & LOSS_fn ##
###########################
# model = DarknetParser(args.config, args.weight)
model = DarknetParser(args.config, args.weight).to(device)
model = torch.nn.DataParallel(model)
loss_function = YOLOv3Loss()


parse from 'C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov4.cfg'
done

load weights from : 'C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov4.weights'
Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
0 convolutional load weights : [0.004]/[245.779] mb
Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
1 convolutional load weights : [0.075]/[245.779] mb
Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
2 convolutional load weights : [0.092]/[245.779] mb
3 route        load weights : [0.092]/[245.779] mb
Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
4 convolutional load weights : [0.108]/[245.779] mb
Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
5 convolutional load weights : [0.117]/[245.779] mb
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
6 convolutional load weights : [0.188]/[245.779] mb
7 shortcut     load weights : [0.188]/[245.779] mb


In [7]:

optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_option["OPTIMIZER"]["LR"])
optimizer_option["OPTIMIZER"]["ITERS_PER_EPOCH"] = len(train_dataset) // optimizer_option["OPTIMIZER"]["BATCH_SIZE"]


In [8]:

logger = SummaryWriter()

if not os.path.isdir(args.weight_save_dir):
    os.makedirs(args.weight_save_dir)



In [9]:

epochs = optimizer_option["OPTIMIZER"]["EPOCHS"]
for epoch in range(epochs):
    ###########
    ## TRAIN ##
    ###########
    train(
            model,
            train_loader,
            loss_function,
            optimizer,
            dataset_option,
            model_option,
            device,
            epoch,
            logger,
            )

    ###########
    ## VALID ##
    ###########
    acc = valid(
                model,
                train_loader,
                model_option,
                device,
                epoch,
                logger
                )

    print(f"Epoch: ({epoch + 1}/{epochs}) . . . [acc: {acc:.2f}]")
    save_checkpoint(epoch,
                    acc,
                    model,
                    optimizer,
                    # scheduler,
                    # scaler,
                    path=args.weight_save_dir
                    )

RuntimeError: Index put requires the source and destination dtypes match, got Double for the destination and Float for the source.